# Imports

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
import category_encoders as ce #pip install category_encoders

# Funciones

# Train

In [ ]:
train = pd.read_csv('train/train.limpio.csv')

## keyword

In [ ]:
ce_bin_keyword = ce.BinaryEncoder(cols = ['keyword'], drop_invariant=True)
keyword_encoded = ce_bin_keyword.fit_transform(train['keyword'])

In [ ]:
keyword_encoded

## location

In [ ]:
ce_bin_location = ce.BinaryEncoder(cols = ['location'])
location_encoded = ce_bin_location.fit_transform(train['location'])

In [ ]:
location_encoded

## text

## len_text

In [ ]:
pos_col_text = train.columns.get_loc('text')+1
train.insert(loc=pos_col_text, column='len_text', value=train['text'].iloc[:].str.len())
train.head(10)

## join

## Resultado

## Guardado del dataframe

In [ ]:
train.tocsv('train/train.csv')

# Test